In [1]:
import csv
import os
import sys
import numpy as np
import torch
import pickle
import copy
import pandas as pd

from logging import Logger
from typing import List
from tqdm import trange

from torch.optim.lr_scheduler import ExponentialLR
from torch_geometric.datasets import QM9

# cd to chempropBayes
%cd /Users/georgelamb/Documents/GitHub/chempropBayes

# import run_training
from chemprop.train.run_training import run_training
from chemprop.args import TrainArgs
from chemprop.data.utils import get_class_sizes, get_data, get_task_names, split_data



/Users/georgelamb/Documents/GitHub/chempropBayes


### Check loss function

Compute through scipy.stats

In [13]:
from scipy import stats

In [14]:
cov = np.array([[3.1, 0],
                [0, 2.2]])**2

output1 = np.array([19.1, 6.2])
target1 = np.array([21.4, 5.1])
output2 = np.array([14.9, 25])
target2 = np.array([13.5, 28])

In [15]:
my_gaussian1 = stats.multivariate_normal(output1, cov)
p1 = my_gaussian1.logpdf(target1) + 0.5*np.log((2*np.pi)**2)
my_gaussian2 = stats.multivariate_normal(output2, cov)
p2 = my_gaussian2.logpdf(target2) + 0.5*np.log((2*np.pi)**2)
np.mean([p1,p2])*-1

2.635841124059944

Compute through loss_sgld

In [19]:
def loss_sgld(output, target, sigma):
    """
    Computes scaled Gaussian log likelihood
    """
    
    exponent = -0.5*torch.sum(
        (target - output)**2/sigma**2
        , 1)

    log_coeff = -torch.sum(torch.log(sigma))
    
    scale = 1 / len(exponent)
    
    return - scale * (log_coeff + exponent).sum()

In [20]:
output = torch.tensor(np.array([[19.1, 6.2],
                                [14.9, 25]]))

sigma = torch.tensor(np.array([3.1, 2.2]))

target = torch.tensor(np.array([[21.4, 5.1],
                                [13.5, 28]]))

In [21]:
loss_sgld(output, target, sigma)

tensor(2.6358, dtype=torch.float64)

### Check optimiser (non preconditioned)

In [2]:
# instantiate args class and load from dict
args = TrainArgs()
args.from_dict({
    'dataset_type': 'regression',
    'data_path': '/Users/georgelamb/Documents/GitHub/chempropBayes/data/QM9.csv'
})

# location for model checkpoints to be saved
args.save_dir = '/Users/georgelamb/Documents/GitHub/chempropBayes/log'

### args (non-model)

# seed for splitting and loading data
args.seed = 0

# data
args.max_data_size = 50000
args.features_path = None
args.features_generator = None

# splitting data
args.split_type = 'random'
args.split_sizes = (0.8, 0.1, 0.1)

# evaluation metric
args.metric = 'mae'

# epochs and logging
args.epochs = 50
args.log_frequency = 800

### args (model)

# seed for random initial weights
args.pytorch_seed = 0

# message passing
args.atom_messages = False
args.undirected = False
args.bias = False
args.hidden_size = 500
args.depth = 5

# FFN
args.ffn_hidden_size = args.hidden_size
args.ffn_num_layers = 3

# shared
args.activation = 'ReLU'


In [3]:
# initial training
args.epochs = 1
args.ensemble_size = 1

args.dropout = 0
args.test_dropout = False
args.swag = False



# sgld
args.samples = 1
args.sgld = True

args.init_log_noise = -2
args.lr_sgld = 0.5
args.norm_sigma_sgld = 0.01

args.batch_size_sgld = 200
args.log_frequency_sgld = 40000/args.batch_size

args.burnin_epochs = 10
args.mix_epochs = 1

results_sgld = run_training(args)
np.savez(args.save_dir+'/results_sgld', results_sgld)


10658it [00:00, 106578.83it/s]

Command line
python /Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py -f /Users/georgelamb/Library/Jupyter/runtime/kernel-9153d363-fe76-4a65-a871-f509e6fac466.json
Args
{'activation': 'ReLU',
 'atom_messages': False,
 'batch_size_sgld': 200,
 'bias': False,
 'block': True,
 'burnin_epochs': 10,
 'c_swag': 0,
 'cache_cutoff': 10000,
 'class_balance': False,
 'config_path': None,
 'cov_mat': False,
 'crossval_index_dir': None,
 'crossval_index_file': None,
 'crossval_index_sets': None,
 'data_path': '/Users/georgelamb/Documents/GitHub/chempropBayes/data/QM9.csv',
 'dataset_type': 'regression',
 'depth': 5,
 'dropout': 0,
 'ensemble_size': 1,
 'epochs': 1,
 'epochs_swag': 0,
 'features_generator': None,
 'features_only': False,
 'features_path': None,
 'features_size': None,
 'ffn_hidden_size': 500,
 'ffn_num_layers': 3,
 'final_lr': 0.0001,
 'folds_file': None,
 'hidden_size': 500,
 'init_log_noise': -2,
 'init_lr': 0.0001,
 'log_frequency': 800,
 'log_frequency_s

46337it [00:00, 78584.30it/s] 
100%|██████████| 50000/50000 [00:03<00:00, 13377.39it/s]


Number of tasks = 12
Splitting data with seed 0
Total size = 50,000 | train size = 40,000 | val size = 5,000 | test size = 5,000
Fitting scaler
Building model 0
MoleculeModel(
  (encoder): MPN(
    (encoder): MPNEncoder(
      (dropout_layer): Dropout(p=0, inplace=False)
      (act_func): ReLU()
      (W_i): Linear(in_features=147, out_features=500, bias=False)
      (W_h): Linear(in_features=500, out_features=500, bias=False)
      (W_o): Linear(in_features=633, out_features=500, bias=True)
    )
  )
  (ffn): Sequential(
    (0): Dropout(p=0, inplace=False)
    (1): Linear(in_features=500, out_features=500, bias=True)
    (2): ReLU()
    (3): Dropout(p=0, inplace=False)
    (4): Linear(in_features=500, out_features=500, bias=True)
    (5): ReLU()
    (6): Dropout(p=0, inplace=False)
    (7): Linear(in_features=500, out_features=12, bias=True)
  )
)
Number of parameters = 1,147,524
Epoch 0
Loss = 6.6179e-03, PNorm = 49.8341, GNorm = 3.9430, lr_0 = 5.5056e-04
Validation mae = 12.860934


add noise:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
add noise:
tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [3.9650e+17, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 5.8606e+21,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]])
add noise:
tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 2.4753e+21,
         0.0000e+00],
        [0.0000e+00,

...

add noise:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
add noise:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
add noise:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
add noise:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan

add noise:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
add noise:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
add noise:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
add noise:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan

add noise:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
add noise:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
add noise:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
add noise:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan

Process Process-18:
Process Process-24:
Process Process-17:
Process Process-21:
Process Process-23:
Process Process-20:
Process Process-19:
Process Process-22:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Applications/anaconda3/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/Applications/anaconda3/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/Applications/anaconda3/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/Applications/anaconda3/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/Applications/anaconda3/lib/python3.7/multiprocessin

KeyboardInterrupt: 